In [195]:
import pdfplumber
import numpy as np
import pandas as pd
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font
import os.path

In [196]:
### FIRST PAGE ###

# page - a Page object from the pdfplumber module
# creates coordinates for address bounding box to extract text from
# returns address_bounding_box - tuple of ints/floats containing coordinates of address bounding box 
def setAddressBoundingBox(page):
    
    #get page height and width for box coordinate calculations
    page_height = page.height
    page_width = page.width
    
    #address box is near second top quarter
    top_half = page.height/2
    top_quarter = page.height/4
    second_quarter = top_half - top_quarter
    
    #coordinates of the address box
    address_box_left = (page.width//2) - 20
    address_box_top = second_quarter - 75
    address_box_right = page.width
    address_box_bottom = (page.height//2) -215
    
    address_bounding_box = (address_box_left,address_box_top,address_box_right,address_box_bottom)
    
    return address_bounding_box


# address_extract_text - str containing extracted text from address box
# returns company_name_text - str containing name of company
def setCompanyName(address_extract_text):
    
    company_name_text = address_extract_text.splitlines()[0]
    
    return company_name_text


# address_extract_text - str containing extracted text from address box
# returns street_text - str containing street name 
def setStreet(address_extract_text):

    street_text = address_extract_text.splitlines()[1]
    
    return street_text


# address_extract_text - str containing extracted text from address box
# returns city_text, state_text, zip_code_text - tuple of str containing city, state, and zip code respectively
def setCityStateZIP(address_extract_text):
    
    city_state_zip_text = address_extract_text.splitlines()[2]
    city_state_zip_text = city_state_zip_text.replace(',', '')
    
    city_text = city_state_zip_text.split(' ')[0]
    state_text =  city_state_zip_text.split(' ')[1]
    zip_code_text =  city_state_zip_text.split(' ')[2]
    
    return city_text, state_text, zip_code_text


# address_extract_text - str containing extracted text from address box
# returns acc_num_text - str containing account number
# NOT USED AS IT TURNS OUT THE ACCOUNT NUMBER IS NOT NEEDED
def setAccNum(address_extract_text): 
    
    acc_num_text = address_extract_text.splitlines()[3]
    acc_num_text = acc_num_text.split('#')[1]
    acc_num_text = acc_num_text.strip()
    
    return acc_num_text

# address_extract_text - str containing extracted text from address box
# returns issued_date_text - str containing issued date 
def setIssuedDate(address_extract_text):
    
    issued_date_text = address_extract_text.splitlines()[4]
    issued_date_text = issued_date_text.split(':')[1]
    issued_date_text = issued_date_text.strip()
    
    return issued_date_text

# page - a Page object from the pdfplumber module
# returns electric_supply_bounding_box - tuple of ints/float containing coordinates of electric supply box
def setElectricSupplyBoundingBox(page):
    
    top_half = page.height/2
    top_quarter = page.height/4
    second_quarter = top_half - top_quarter

    electric_supply_box_top  = second_quarter - 75
    electric_supply_box_bottom = (page.height//2) - 215
    electric_supply_box_left = (page.width//2) - 130
    electric_supply_box_right = (page.width//2)
    
    electric_supply_bounding_box = (electric_supply_box_left,electric_supply_box_top,electric_supply_box_right,electric_supply_box_bottom)
    
    return electric_supply_bounding_box


# page - a Page object from the pdfplumber module
# returns utility_name_text  - str containing name of utility
def setUtilityName(page):
    
    page_extract = page.extract_text(x_tolerance=1, y_tolerance=1)
    utility_name_text = page_extract.splitlines()[-6]
    
    return utility_name_text

### SECOND PAGE ###

# page - a Page object from the pdfplumber module
# creates coordinates for rate bounding box to extract text from
# returns rate_bounding_box - tuple of ints/floats containing coordinates of rate bounding box 
def setElectricRateBoundingBox(page):
    
    rate_box_left = 20
    rate_box_top = (page.width//4) + 5
    rate_box_right = (page.width//2) - 100
    rate_box_bottom = (page.width//4) + 50
    rate_bounding_box = (rate_box_left, rate_box_top, rate_box_right, rate_box_bottom)
    
    return rate_bounding_box


# rate_extract_text - str containing extracted text from rate box
# returns rate_text - str containing rate code
def setRate(rate_extract_text):

    #gets the first line after the word 'Service' and remove the leading whitespace and '-'
    rate_text = rate_extract_text.split('Service')[1].splitlines()[0].replace('-','',1).strip()
    
    if 'TOU -' in rate_text:
        
        #remove TOU and leading '-' for uniformity
        rate_text = rate_text.split('TOU')[1].replace('-','',1).strip()
    
    return rate_text

# page - a Page object from the pdfplumber module
# creates coordinates for usage bounding box to extract text from
# returns usage_bounding_box - tuple of ints/floats containing coordinates of usage bounding box 
def setElectricUsageBoundingBox(page):
    
    usage_box_left = (page.width//2) - 130
    usage_box_top = (page.height//2) - 180
    usage_box_right = (page.width//2) - 20
    usage_box_bottom = (page.height//2) - 150
    usage_bounding_box = (usage_box_left,usage_box_top,usage_box_right,usage_box_bottom)
    
    return usage_bounding_box

# page - a Page object from the pdfplumber module
# usage_bounding_box - tuple of ints/floats containing coordinates of usage bounding box
# returns usage_extract_text - str containing extracted text from usage box

def setUsage(usage_extract_text):
    
    usage_text = usage_extract_text.splitlines()[0]
    
    return usage_text


    

In [197]:
# page - a Page object from the pdfplumber module
# returns gas_supply_bounding_box - tuple of ints/float containing coordinates of electric supply box
def setGasSupplyLowerBoundingBox(page):
    
    top_half = page.height/2
    top_quarter = page.height/4
    second_quarter = top_half - top_quarter

    gas_supply_box_top  = second_quarter + 80
    gas_supply_box_bottom = (page.height//2) + 20
    gas_supply_box_left = (page.width//2) - 130
    gas_supply_box_right = (page.width//2)
    
    gas_supply_bounding_box = (gas_supply_box_left,gas_supply_box_top,gas_supply_box_right,gas_supply_box_bottom)
    
    return gas_supply_bounding_box


# page - a Page object from the pdfplumber module
# creates coordinates for usage bounding box to extract text from
# returns usage_bounding_box - tuple of ints/floats containing coordinates of usage bounding box 
def setGasUsageRightBoundingBox(page):
    
    usage_box_left = page.width - 90
    usage_box_top = (page.height//2) - 180
    usage_box_right = page.width - 20
    usage_box_bottom = (page.height//2) - 145
    usage_bounding_box = (usage_box_left,usage_box_top,usage_box_right,usage_box_bottom)
    
    return usage_bounding_box


In [198]:
def setGasRateRightBoundingBox(page):
    
    rate_box_left = (page.width//2) + 20
    rate_box_top = (page.width//4) + 5
    rate_box_right = (page.width) - 100
    rate_box_bottom = (page.width//4) + 50
    rate_bounding_box = (rate_box_left, rate_box_top, rate_box_right, rate_box_bottom)
    
    return rate_bounding_box

In [199]:
#for no supplier only
def setGasOnlyUsage(usage_extract_text):
       
    usage_text = usage_extract_text.splitlines()[1]
    
    return usage_text 

In [200]:
def setGasRateNoSuppBoundingBox(page):
    
    rate_box_left = (page.width//2) - 300
    rate_box_top = 60
    rate_box_right = (page.width//2) - 30
    rate_box_bottom = 90
    rate_bounding_box = (rate_box_left,rate_box_top,rate_box_right,rate_box_bottom)
    
    return rate_bounding_box

# second_page.crop(setGasRateUpperBoundingBox(second_page)).extract_text(x_tolerance = 1)

In [201]:
def setGasUsageNoSuppBoundingBox(page):
    usage_box_left = (page.width//2) - 60
    usage_box_top = 100
    usage_box_right = (page.width//2) - 5
    usage_box_bottom = 130
    usage_bounding_box = (usage_box_left,usage_box_top,usage_box_right,usage_box_bottom)
    
    return usage_bounding_box

# usage = second_page.crop(usage_bounding_box).extract_text(x_tolerance = 1)
# usage.splitlines()[1]

In [202]:
def setGasRateSuppBoundingBox(page):
    rate_box_left = (page.width//2) - 300
    rate_box_top = 160
    rate_box_right = (page.width//2) - 30
    rate_box_bottom = 210
    rate_bounding_box = (rate_box_left,rate_box_top,rate_box_right,rate_box_bottom)
    
    return rate_bounding_box

In [203]:
def setGasUsageSuppBoundingBox(page):
    
    usage_box_left = (second_page.width//2) - 80
    usage_box_top = 220
    usage_box_right = (second_page.width//2)
    usage_box_bottom = 250
    usage_bounding_box = (usage_box_left,usage_box_top,usage_box_right,usage_box_bottom)
    
    return usage_bounding_box

In [204]:
### COMBINING DUPLICATE FUNCTIONS FOR GAS AND ELECTRIC HERE ###

def defaultExtractText(page, bounding_box):
    
    extracted_text = page.crop(bounding_box).extract_text(x_tolerance=1)
    
    return extracted_text
    
def setSupplier(extracted_text):
    
    supplier_text = extracted_text.splitlines()[1]
    
    return supplier_text

def setChoiceID(extracted_text):
    
    choice_id = extracted_text.split('Choice ID:')[1].strip()
    
    return choice_id

In [213]:
file_name = input("Name of PDF file: ")

# file_name = 'PII Example 1'
#with pdfplumber.open('Bills/'+ file_name + '.pdf') as pdf:

#initializing variables outside scope of open file
utility_name = ''
company_name = ''
street = ''
city = ''
state = ''
zip_code = ''
issued_date = ''
electric_supplier = ''
electric_choice_id = ''
gas_supplier = ''
gas_choice_id = ''
electric_rate = ''
electric_usage = ''
gas_rate = ''
gas_usage = ''
bill_dict = {}

supplier_present = ''

electric_supplier_present = ''
gas_supplier_present = ''

#adding logic for different cases

print('What type of bill is this?')
print('e - electricity only    g - gas only    eg - electricity and gas ')
bill_type = input('bill type: ')

while (bill_type != 'e') and (bill_type != 'g') and (bill_type != 'eg'):
    print('Invalid choice. Try again:')
    bill_type = input('bill type: ')

# print('Is there a supplier on the bill?')
# print('yes    no')
# supplier_present = input().lower()
    
if (bill_type == 'e') or (bill_type =='g'):
    print('Is there a supplier on the bill?')
    print('yes    no')
    supplier_present = input().lower()
    while (supplier_present != 'yes') and (supplier_present != 'no'):
        print('Invalid choice. Try again:')
        supplier_present = input()
        
if (bill_type =='eg'):
    print('Is there an electric supplier on the bill?')
    print('yes    no')
    electric_supplier_present = input().lower()
    while (electric_supplier_present != 'yes') and (electric_supplier_present != 'no'):
        print('Invalid choice. Try again:')
        electric_supplier_present = input()
    
    print('Is there a gas supplier on the bill?')
    print('yes    no')
    gas_supplier_present = input().lower()
    while (gas_supplier_present != 'yes') and (gas_supplier_present != 'no'):
        print('Invalid choice. Try again:')
        gas_supplier_present = input()


# if(bill_type == 'e'):
#     gas_supplier_present = 'no'
#     electric_supplier_present = supplier_present
    
# elif(bill_type == 'g'):
#     electric_supplier_present = 'no'
#     gas_supplier_present = supplier_present
    
# else: 
    
    




Name of PDF file: 2.18.20 - G
What type of bill is this?
e - electricity only    g - gas only    eg - electricity and gas 
bill type: g
Is there a supplier on the bill?
yes    no
yes


In [206]:
if (bill_type == 'e') and (supplier_present == 'no'): 
     with pdfplumber.open(f'Bills/{file_name}.pdf') as pdf:
        first_page = pdf.pages[0]
        second_page = pdf.pages[1]

        #extracting from address box
        address_extract = defaultExtractText(first_page, setAddressBoundingBox(first_page))

        company_name = setCompanyName(address_extract)
        street = setStreet(address_extract)
        city, state, zip_code = setCityStateZIP(address_extract)
        issued_date = setIssuedDate(address_extract)

        #extracting from electric supply box
        electric_supply_extract = defaultExtractText(first_page, setElectricSupplyBoundingBox(first_page))
        electric_supplier = np.nan
        electric_choice_id = setChoiceID(electric_supply_extract) 
        
        #extracting utility name from the bottom of the page
        utility_name = setUtilityName(first_page)

        #extracting from electric_rate box
        electric_rate_extract = defaultExtractText(second_page, setElectricRateBoundingBox(second_page))
        electric_rate = setRate(electric_rate_extract)
        print(electric_rate)

        #extracting from electric_usage
        electric_usage_extract = defaultExtractText(second_page, setElectricUsageBoundingBox(second_page))
        electric_usage = setUsage(electric_usage_extract)
        
        gas_supplier = np.nan
        gas_choice_id = np.nan
        gas_rate = np.nan
        gas_usage = np.nan

bill_dict = dict(utility = utility_name, issued_date = issued_date, company = company_name, 
                street = street, city = city, state = state, zip_code = zip_code, 
                electric_choice_id = electric_choice_id, electric_rate_code = electric_rate, 
                electric_supplier = electric_supplier, electric_usage = electric_usage,
                gas_supplier = gas_supplier, gas_choice_id = gas_choice_id, gas_rate_code = gas_rate, 
                gas_usage = gas_usage)

    #10.22.20 E, 11.18.20 E format looks slightly different so doesn't work

# bill_dict
        

In [207]:
if (bill_type == 'e') and (supplier_present == 'yes'): 
    with pdfplumber.open(f'Bills/{file_name}.pdf') as pdf:
        first_page = pdf.pages[0]
        second_page = pdf.pages[1]

        #extracting from address box
        address_extract = defaultExtractText(first_page, setAddressBoundingBox(first_page))

        company_name = setCompanyName(address_extract)
        street = setStreet(address_extract)
        city, state, zip_code = setCityStateZIP(address_extract)
        issued_date = setIssuedDate(address_extract)

        #extracting from electric supply box
        electric_supply_extract = defaultExtractText(first_page, setElectricSupplyBoundingBox(first_page))

        electric_supplier = setSupplier(electric_supply_extract)
        electric_choice_id = setChoiceID(electric_supply_extract)

        #extracting utility name from the bottom of the page
        utility_name = setUtilityName(first_page)

        #extracting from rate box
        rate_extract = defaultExtractText(second_page, setElectricRateBoundingBox(second_page))
        electric_rate = setRate(rate_extract)
        print(electric_rate)

        #extracting from usage
        usage_extract = defaultExtractText(second_page, setElectricUsageBoundingBox(second_page))
        electric_usage = setUsage(usage_extract)
        
        gas_supplier = np.nan
        gas_choice_id = np.nan
        gas_rate = np.nan
        gas_usage = np.nan


bill_dict = dict(utility = utility_name, issued_date = issued_date, company = company_name, 
                street = street, city = city, state = state, zip_code = zip_code, 
                electric_choice_id = electric_choice_id, electric_rate_code = electric_rate, 
                electric_supplier = electric_supplier, electric_usage = electric_usage,
                gas_supplier = gas_supplier, gas_choice_id = gas_choice_id, gas_rate_code = gas_rate, gas_usage = gas_usage)

    #10.22.20 E, 11.18.20 E format looks slightly different so doesn't work

# bill_dict


In [208]:
if (bill_type == 'eg') and (electric_supplier_present == 'no') and (gas_supplier_present == 'no'):
    with pdfplumber.open(f'Bills/{file_name}.pdf') as pdf:
        first_page = pdf.pages[0]
        second_page = pdf.pages[1]
        
        #extracting from address box
        address_extract = defaultExtractText(first_page, setAddressBoundingBox(first_page))

        company_name = setCompanyName(address_extract)
        street = setStreet(address_extract)
        city, state, zip_code = setCityStateZIP(address_extract)
        issued_date = setIssuedDate(address_extract)
        
        #extracting from electric supply box
        electric_supply_extract = defaultExtractText(first_page, setElectricSupplyBoundingBox(first_page))
        electric_supplier = np.nan
        electric_choice_id = setChoiceID(electric_supply_extract) 
        
        #extracting from gas supply box
        gas_supply_extract = defaultExtractText(first_page, setGasSupplyLowerBoundingBox(first_page))
        gas_choice_id = setChoiceID(gas_supply_extract)
        gas_supplier = np.nan
        
        #extracting electric rate code from second page
        rate_extract = defaultExtractText(second_page, setElectricRateBoundingBox(second_page))
        electric_rate = setRate(rate_extract)
        
        electric_usage_extract = defaultExtractText(second_page, setElectricUsageBoundingBox(second_page))
        electric_usage = setUsage(electric_usage_extract)
        
        #extracting from gas details on second page
        gas_usage_extract = defaultExtractText(second_page, setGasUsageRightBoundingBox(second_page))
        gas_usage = setUsage(gas_usage_extract)
        
        gas_rate = setRate(defaultExtractText(second_page, setGasRateRightBoundingBox(second_page)))
        
        #extracting utility name from the bottom of the page
        utility_name = setUtilityName(first_page)

                
bill_dict = dict(utility = utility_name, issued_date = issued_date, company = company_name, 
                street = street, city = city, state = state, zip_code = zip_code, 
                electric_choice_id = electric_choice_id, electric_rate_code = electric_rate, 
                electric_supplier = electric_supplier, electric_usage = electric_usage,
                gas_supplier = gas_supplier, gas_choice_id = gas_choice_id, gas_rate_code = gas_rate, gas_usage = gas_usage)


In [209]:
if (bill_type == 'eg') and (electric_supplier_present == 'yes') and (gas_supplier_present == 'yes'):
    with pdfplumber.open(f'Bills/{file_name}.pdf') as pdf:
        first_page = pdf.pages[0]
        second_page = pdf.pages[1]
        
        #extracting from address box
        address_extract = defaultExtractText(first_page, setAddressBoundingBox(first_page))

        company_name = setCompanyName(address_extract)
        street = setStreet(address_extract)
        city, state, zip_code = setCityStateZIP(address_extract)
        issued_date = setIssuedDate(address_extract)
        
        #extracting from electric supply box
        electric_supply_extract = defaultExtractText(first_page, setElectricSupplyBoundingBox(first_page))
        electric_supplier = setSupplier(electric_supply_extract)
        electric_choice_id = setChoiceID(electric_supply_extract)
        
        #extracting from gas supply box
        gas_supply_extract = defaultExtractText(first_page, setGasSupplyLowerBoundingBox(first_page))
        gas_supplier = setSupplier(gas_supply_extract)
        gas_choice_id = setChoiceID(gas_supply_extract)
        
        #extracting electric rate code from second page
        rate_extract = defaultExtractText(second_page, setElectricRateBoundingBox(second_page))
        electric_rate = setRate(rate_extract)
        
        electric_usage_extract = defaultExtractText(second_page, setElectricUsageBoundingBox(second_page))
        electric_usage = setUsage(electric_usage_extract)
        
        #extracting from gas details on second page
        gas_usage_extract = defaultExtractText(second_page, setGasUsageRightBoundingBox(second_page))
        gas_usage = setUsage(gas_usage_extract)
        
        gas_rate = setRate(defaultExtractText(second_page, setGasRateRightBoundingBox(second_page)))
        
        #extracting utility name from the bottom of the page
        utility_name = setUtilityName(first_page)
        
bill_dict = dict(utility = utility_name, issued_date = issued_date, company = company_name, 
                street = street, city = city, state = state, zip_code = zip_code, 
                electric_choice_id = electric_choice_id, electric_rate_code = electric_rate, 
                electric_supplier = electric_supplier, electric_usage = electric_usage,
                gas_supplier = gas_supplier, gas_choice_id = gas_choice_id, gas_rate_code = gas_rate, gas_usage = gas_usage)


In [210]:
if (bill_type == 'eg') and (electric_supplier_present == 'yes') and (gas_supplier_present == 'no'):
    with pdfplumber.open(f'Bills/{file_name}.pdf') as pdf:
        first_page = pdf.pages[0]
        second_page = pdf.pages[1]
        
        #extracting from address box
        address_extract = defaultExtractText(first_page, setAddressBoundingBox(first_page))

        company_name = setCompanyName(address_extract)
        street = setStreet(address_extract)
        city, state, zip_code = setCityStateZIP(address_extract)
        issued_date = setIssuedDate(address_extract)
        
        #extracting from electric supply box
        electric_supply_extract = defaultExtractText(first_page, setElectricSupplyBoundingBox(first_page))
        electric_supplier = setSupplier(electric_supply_extract)
        electric_choice_id = setChoiceID(electric_supply_extract)
        
        #extracting from gas supply box
        gas_supply_extract = defaultExtractText(first_page, setGasSupplyLowerBoundingBox(first_page))
        gas_supplier = np.nan
        gas_choice_id = setChoiceID(gas_supply_extract)
        
        #extracting electric rate code from second page
        rate_extract = defaultExtractText(second_page, setElectricRateBoundingBox(second_page))
        electric_rate = setRate(rate_extract)
        
        electric_usage_extract = defaultExtractText(second_page, setElectricUsageBoundingBox(second_page))
        electric_usage = setUsage(electric_usage_extract)
        
        #extracting from gas details on second page
        gas_usage_extract = defaultExtractText(second_page, setGasUsageRightBoundingBox(second_page))
        gas_usage = setUsage(gas_usage_extract)
        
        gas_rate = setRate(defaultExtractText(second_page, setGasRateRightBoundingBox(second_page)))
        
        #extracting utility name from the bottom of the page
        utility_name = setUtilityName(first_page)
        
bill_dict = dict(utility = utility_name, issued_date = issued_date, company = company_name, 
                street = street, city = city, state = state, zip_code = zip_code, 
                electric_choice_id = electric_choice_id, electric_rate_code = electric_rate, 
                electric_supplier = electric_supplier, electric_usage = electric_usage,
                gas_supplier = gas_supplier, gas_choice_id = gas_choice_id, gas_rate_code = gas_rate, gas_usage = gas_usage)


In [211]:
if (bill_type == 'eg') and (electric_supplier_present == 'no') and (gas_supplier_present == 'yes'):
    with pdfplumber.open(f'Bills/{file_name}.pdf') as pdf:
        first_page = pdf.pages[0]
        second_page = pdf.pages[1]
        
        #extracting from address box
        address_extract = defaultExtractText(first_page, setAddressBoundingBox(first_page))

        company_name = setCompanyName(address_extract)
        street = setStreet(address_extract)
        city, state, zip_code = setCityStateZIP(address_extract)
        issued_date = setIssuedDate(address_extract)
        
        #extracting from electric supply box
        electric_supply_extract = defaultExtractText(first_page, setElectricSupplyBoundingBox(first_page))
        electric_supplier = np.nan
        electric_choice_id = setChoiceID(electric_supply_extract)
        
        #extracting from gas supply box
        gas_supply_extract = defaultExtractText(first_page, setGasSupplyLowerBoundingBox(first_page))
        gas_supplier = setSupplier(gas_supply_extract)
        gas_choice_id = setChoiceID(gas_supply_extract)
        
        #extracting electric rate code from second page
        rate_extract = defaultExtractText(second_page, setElectricRateBoundingBox(second_page))
        electric_rate = setRate(rate_extract)
        
        electric_usage_extract = defaultExtractText(second_page, setElectricUsageBoundingBox(second_page))
        electric_usage = setUsage(electric_usage_extract)
        
        #extracting from gas details on second page
        gas_usage_extract = defaultExtractText(second_page, setGasUsageRightBoundingBox(second_page))
        gas_usage = setUsage(gas_usage_extract)
        
        gas_rate = setRate(defaultExtractText(second_page, setGasRateRightBoundingBox(second_page)))
        
        #extracting utility name from the bottom of the page
        utility_name = setUtilityName(first_page)
        
bill_dict = dict(utility = utility_name, issued_date = issued_date, company = company_name, 
                street = street, city = city, state = state, zip_code = zip_code, 
                electric_choice_id = electric_choice_id, electric_rate_code = electric_rate, 
                electric_supplier = electric_supplier, electric_usage = electric_usage,
                gas_supplier = gas_supplier, gas_choice_id = gas_choice_id, gas_rate_code = gas_rate, gas_usage = gas_usage)


In [214]:
if (bill_type == 'g') and (supplier_present == 'no'):
    with pdfplumber.open(f'Bills/{file_name}.pdf') as pdf:
        first_page = pdf.pages[0]
        second_page = pdf.pages[1]
        
        #extracting from address box
        address_extract = defaultExtractText(first_page, setAddressBoundingBox(first_page))

        company_name = setCompanyName(address_extract)
        street = setStreet(address_extract)
        city, state, zip_code = setCityStateZIP(address_extract)
        issued_date = setIssuedDate(address_extract)
        
        electric_supplier = np.nan
        electric_choice_id = np.nan
        electric_rate = np.nan
        electric_usage = np.nan
        
        #extracting from gas supply box
        gas_supply_extract = defaultExtractText(first_page, setElectricSupplyBoundingBox(first_page))
        gas_choice_id = setChoiceID(gas_supply_extract)
        print(gas_choice_id)
        
         #extracting electric rate code from second page
#         rate_extract = extractRateBoxText(second_page, setGasRateUpperBoundingBox(second_page))
#         gas_rate = setRate(rate_extract)
        gas_supplier = np.nan
    
        gas_usage_extract = defaultExtractText(second_page, setGasUsageNoSuppBoundingBox(second_page))
        gas_usage = setGasOnlyUsage(gas_usage_extract)
        print(gas_usage)
        

#         gas_rate = setRate(defaultExtractText(second_page, setGasRateUpperBoundingBox(second_page)))
        gas_rate = setRate(defaultExtractText(second_page, setGasRateNoSuppBoundingBox(second_page)))
        print(gas_rate)
        
        #extracting utility name from the bottom of the page
        utility_name = setUtilityName(first_page)

bill_dict = dict(utility = utility_name, issued_date = issued_date, company = company_name, 
                street = street, city = city, state = state, zip_code = zip_code, 
                electric_choice_id = electric_choice_id, electric_rate_code = electric_rate, 
                electric_supplier = electric_supplier, electric_usage = electric_usage,
                gas_supplier = gas_supplier, gas_choice_id = gas_choice_id, gas_rate_code = gas_rate, gas_usage = gas_usage)

        
        

In [215]:
if (bill_type == 'g') and (supplier_present == 'yes'):
    with pdfplumber.open(f'Bills/{file_name}.pdf') as pdf:
        first_page = pdf.pages[0]
        second_page = pdf.pages[1]
        
        #extracting from address box
        address_extract = defaultExtractText(first_page, setAddressBoundingBox(first_page))

        company_name = setCompanyName(address_extract)
        street = setStreet(address_extract)
        city, state, zip_code = setCityStateZIP(address_extract)
        issued_date = setIssuedDate(address_extract)
        
        electric_supplier = np.nan
        electric_choice_id = np.nan
        electric_rate = np.nan
        electric_usage = np.nan
        
        #extracting from gas supply box
        
        gas_supply_extract = defaultExtractText(first_page, setElectricSupplyBoundingBox(first_page))
#         gas_choice_id = setGasChoiceID(gas_supply_extract)
        gas_choice_id = setChoiceID(gas_supply_extract)
        gas_supplier = setSupplier(gas_supply_extract)
        print(gas_supplier)
        print(gas_choice_id)
        
        gas_rate = setRate(defaultExtractText(second_page, setGasRateSuppBoundingBox(second_page)))
        print(gas_rate)
        
        gas_usage = defaultExtractText(second_page, setGasUsageSuppBoundingBox(second_page)).splitlines()[0]
        print(gas_usage)
        
        #extracting utility name from the bottom of the page
        utility_name = setUtilityName(first_page)

bill_dict = dict(utility = utility_name, issued_date = issued_date, company = company_name, 
                street = street, city = city, state = state, zip_code = zip_code, 
                electric_choice_id = electric_choice_id, electric_rate_code = electric_rate, 
                electric_supplier = electric_supplier, electric_usage = electric_usage,
                gas_supplier = gas_supplier, gas_choice_id = gas_choice_id, gas_rate_code = gas_rate, gas_usage = gas_usage)

        
        

BGE
6505952885
Schedule C
20095


In [216]:
bill_df = pd.DataFrame([bill_dict])
bill_df.columns = [col.replace('_',' ') for col in bill_df]
bill_df.columns = [col.title() for col in bill_df]
bill_df.rename(columns = {
    'Electric Choice Id': 'Electric Choice ID',
    'Electric Usage' : 'Electric Usage (kWh)',
    'Gas Choice Id' : 'Gas Choice ID',
    'Gas Usage' : 'Gas Usage (therms)',
    'Zip Code' : 'ZIP Code'
}, inplace = True)

bill_df.fillna('N/A', inplace = True)

bill_df

,Utility,Issued Date,Company,Street,City,State,ZIP Code,Electric Choice ID,Electric Rate Code,Electric Supplier,Electric Usage (kWh),Gas Supplier,Gas Choice ID,Gas Rate Code,Gas Usage (therms)
0,BGE,"February 18, 2020",Pharmaceutics International Inc,10819 Gilroy Rd *Suite 100,Cockeysville,MD,21031,N/A,N/A,N/A,N/A,BGE,6505952885,Schedule C,20095


In [218]:
if os.path.isfile('bill_data.xlsx') == True:
    wb = openpyxl.load_workbook('bill_data.xlsx')
    ws = wb.active
    
    for r in dataframe_to_rows(bill_df, index=False, header=False):
        ws.append(r)
    

else:
    wb = openpyxl.Workbook()
    ws = wb.active
    
    for r in dataframe_to_rows(bill_df, index=False, header=True):
        ws.append(r)

# ws['A1':'O1'].font = Font(bold = True)

for i in range(0, 15):
    ws.cell(row = 1, column = i+1).font = Font(bold = True)
        
wb.save('bill_data.xlsx')
    
bill_df

,Utility,Issued Date,Company,Street,City,State,ZIP Code,Electric Choice ID,Electric Rate Code,Electric Supplier,Electric Usage (kWh),Gas Supplier,Gas Choice ID,Gas Rate Code,Gas Usage (therms)
0,BGE,"February 18, 2020",Pharmaceutics International Inc,10819 Gilroy Rd *Suite 100,Cockeysville,MD,21031,N/A,N/A,N/A,N/A,BGE,6505952885,Schedule C,20095


In [103]:
bill_df.to_excel(f'{file_name}.xlsx')